## Imports

In [1]:
from huggingface_hub import hf_hub_download
from IPython.display import display, Audio
from kokoro import KModel, KPipeline
import soundfile as sf

## Download model

In [2]:
REPO_ID = "hexgrad/Kokoro-82M"

In [3]:
model_path = hf_hub_download(
    repo_id=REPO_ID,
    filename=KModel.MODEL_NAMES[REPO_ID],
    local_dir="../models/kokoro",
    force_download=False,  # Set to True to force redownload even if the file exists
)
model_path

'../models/kokoro/kokoro-v1_0.pth'

## Initialize model

In [4]:
%%time
model = KModel(repo_id=REPO_ID, model=model_path)

/Users/csr95/anaconda3/envs/read-aloud-ai/lib/python3.11/site-packages/torch/nn/modules/rnn.py:83: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
/Users/csr95/anaconda3/envs/read-aloud-ai/lib/python3.11/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


CPU times: user 1.59 s, sys: 918 ms, total: 2.5 s
Wall time: 4.11 s


## Initialize pipeline

KPipeline is a language-aware support class with 2 main responsibilities:
1. Perform language-specific G2P (Grapheme-to-Phoneme), mapping (and chunking) text -> phonemes
2. Manage and store voices, lazily downloaded from HF if needed

You are expected to have one KPipeline per language. If you have multiple KPipelines, you should reuse one KModel instance across all of them.

By default, KPipeline will automatically initialize its own KModel (`model=True`). With `model=False` we construct a "quiet" KPipeline, which means that KPipeline yields (graphemes, phonemes, None) without generating any audio. You can use this to phonemize and chunk your text in advance.

A "loud" KPipeline _with_ a model yields (graphemes, phonemes, audio).

Args:
    lang_code: Language code for G2P processing
    model: KModel instance, True to create new model, False for no model (default: True)
    trf: Whether to use transformer-based G2P (default: False)
    device: Override default device selection ('cuda' or 'cpu', or None for auto)
        If None, will auto-select cuda if available
        If 'cuda' and not available, will explicitly raise an error

### Language codes

```python
LANG_CODES = dict(
    # pip install misaki[en]
    a='American English',
    b='British English',

    # espeak-ng
    e='es',
    f='fr-fr',
    h='hi',
    i='it',
    p='pt-br',

    # pip install misaki[ja]
    j='Japanese',

    # pip install misaki[zh]
    z='Mandarin Chinese',
)
```

In [5]:
%%time
pipeline = KPipeline(lang_code="a", repo_id=REPO_ID, model=model, device="cpu")

CPU times: user 1.13 s, sys: 213 ms, total: 1.34 s
Wall time: 4.12 s


## Generate, display, and save audio files in a loop

See voices samples [here](https://huggingface.co/onnx-community/Kokoro-82M-v1.0-ONNX#voicessamples).

```python

def pipeline(
    text: str | List[str],
    voice: str | None = None,
    speed: float | ((int) -> float) = 1,
    split_pattern: str | None = r'\n+',
    model: KModel | None = None
) -> Generator[Result, None, None]
```

In [6]:
text = '''
[Kokoro](/kˈOkəɹO/) is an open-weight TTS model with 82 million parameters. Despite its lightweight architecture, it delivers comparable quality to larger models while being significantly faster and more cost-efficient. With Apache-licensed weights, [Kokoro](/kˈOkəɹO/) can be deployed anywhere from production environments to personal projects.
'''
generator = pipeline(text, voice='af_heart')

In [ ]:
%%time

for i, (graphemes, phonemes, audio) in enumerate(generator):
    print(
        "-------------------------------\n"
        f"Audio {i}:\n"
        f"  Graphemes: {graphemes}\n"
        f"  Phonemes: {phonemes}\n"
    )
    display(
        Audio(
            data=audio,
            rate=24000,  # Default sample rate for Kokoro. Increasing this value will accelerate the playback speed
            autoplay=i==0  # Autoplays the first audio and not the others
        )
    )
    # Save the audio to a file
    # sf.write(f"../data/output_audio/{i}.wav", audio, 24000)

-------------------------------
Audio 0:
  Graphemes: Kokoro is an open-weight TTS model with 82 million parameters. Despite its lightweight architecture, it delivers comparable quality to larger models while being significantly faster and more cost-efficient. With Apache-licensed weights, Kokoro can be deployed anywhere from production environments to personal projects.
  Phonemes: kˈOkəɹO ɪz ɐn ˈOpᵊnwˌAt tˌitˌiˈɛs mˈɑdᵊl wɪð ˈATi tˈu mˈɪljᵊn pəɹˈæməTəɹz. dəspˈIt ɪts lˈItwˌAt ˈɑɹkətˌɛkʧəɹ, ɪt dəlˈɪvəɹz kˈɑmpəɹəbᵊl kwˈɑləTi tə lˈɑɹʤəɹ mˈɑdᵊlz wˌIl bˈiɪŋ səɡnˈɪfəkəntli fˈæstəɹ ænd mˈɔɹ kˈɔstəfˌɪʃənt. wˌɪð əpˌæʧilˈIsᵊnst wˈAts, kˈOkəɹO kæn bi dəplˈYd ˈɛniwˌɛɹ fɹʌm pɹədˈʌkʃən ənvˈIɹənmᵊnts tə pˈɜɹsᵊnəl pɹˈɑʤˌɛkts.



CPU times: user 29.8 s, sys: 2.55 s, total: 32.4 s
Wall time: 20.8 s
